# Nayan Sandbox

In [181]:
%matplotlib inline

import re
import csv
import nltk
import operator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from nltk.stem.porter import PorterStemmer

In [182]:
tweet_sensing_data = pd.read_csv('Earthquake sensing tweets.csv',header=0,delimiter=',')

In [183]:
tweet_sensing_data

,Tweet Number,Tweet,Class
0,1,Huge one.... Shook my whole bed... #earthquake,1
1,2,"#earthquake ws an awful alarm to wake up to,wi...",1
2,3,if u encounter #earthquake then retweet and if...,1
3,4,Felt Earthquake For The First Time.. It was fo...,1
4,5,#earthquake I felt it just few minutes back he...,1
5,6,Did anyone else just feel that earthquake? #Ne...,1
6,7,Was it an earthquake? Delhi guys? Anyone?,1
7,8,EARTHQUAKE!!!,1
8,9,"That's true, high mag #earthquake in Gurgaon a...",1
9,10,Something happening to me or is it #earthquake,1


In [184]:
def find_url(tweet):
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet)
    if len(urls) !=0:
        return 1
    else:
        return 0


In [185]:
def has_magnitude(tweet):
    decimal = re.findall("\d+\.\d\s", tweet)
    if len(decimal) != 0:
        return 1
    else:
        return 0

In [186]:
def find_position(val):
    for i in range(len(val)):
        if val[i].lower().find('earthquake') != -1:
            return i
    return -1

def Feature_extraction_A(X):
    X['total_words'] = X.Tweet.str.split(' ').apply(len)
    X['position_query_word'] = X.Tweet.str.split().apply(find_position)

In [187]:
tweet_sensing_data['has_magn']= tweet_sensing_data.Tweet.apply(has_magnitude)
tweet_sensing_data['has_url']= tweet_sensing_data.Tweet.apply(find_url)
#tweet_sensing_data

In [188]:
Feature_extraction_A(tweet_sensing_data)

In [150]:
#tweet_sensing_data 

In [189]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
import sklearn.metrics

In [190]:
dataArray = tweet_sensing_data.values
X = dataArray[:,3:]
X = np.array(X, dtype='float')
y = dataArray[:,2]
y = np.array(y, dtype='float')

from sklearn import preprocessing
X = preprocessing.scale(X)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [172]:
C_range = np.logspace(-2, 10, 10, base=2)
gamma_range = np.logspace(-9, 1, 5, base=2)
k_options = ['linear','poly','rbf']
params_grids = dict(gamma=gamma_range, C=C_range, kernel=k_options)
grid = GridSearchCV(SVC(), param_grid=params_grids, cv=10, n_jobs=10)
grid.fit(X_train,y_train)

print grid.best_params_
#grid.grid_scores_

GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=10,
       param_grid={'kernel': ['linear', 'poly', 'rbf'], 'C': array([  2.50000e-01,   6.29961e-01,   1.58740e+00,   4.00000e+00,
         1.00794e+01,   2.53984e+01,   6.40000e+01,   1.61270e+02,
         4.06375e+02,   1.02400e+03]), 'gamma': array([  1.95312e-03,   1.10485e-02,   6.25000e-02,   3.53553e-01,
         2.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

{'kernel': 'rbf', 'C': 161.26989438654363, 'gamma': 0.35355339059327379}


In [192]:
#grid.grid_scores_

In [197]:
y_pred = SVC(kernel=grid.best_params_['kernel'] ,C=grid.best_params_['C'], 
             gamma=grid.best_params_['gamma']).fit(X_train[:,2:],y_train).predict(X_test[:,2:])

sklearn.metrics.accuracy_score(y_test, y_pred)
sklearn.metrics.precision_score(y_test, y_pred)
sklearn.metrics.recall_score(y_test, y_pred)
sklearn.metrics.f1_score(y_test, y_pred)

0.80981595092024539

0.81428571428571428

0.76000000000000001

0.78620689655172415

In [198]:
from sklearn.ensemble import RandomForestClassifier

#choosing the best set of hyperparameters
param_grid =  { "n_estimators" :[10,20,30,40,50],
                "max_features": [1,2,3,4]}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=30, n_jobs=10)
grid_search.fit(X_train, y_train)

print grid_search.best_params_
#grid_search.grid_scores_

GridSearchCV(cv=30, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=10,
       param_grid={'n_estimators': [10, 20, 30, 40, 50], 'max_features': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

{'max_features': 1, 'n_estimators': 10}


In [199]:
grid_search.grid_scores_

[mean: 0.91217, std: 0.06377, params: {'max_features': 1, 'n_estimators': 10},
 mean: 0.90755, std: 0.06333, params: {'max_features': 1, 'n_estimators': 20},
 mean: 0.90293, std: 0.06444, params: {'max_features': 1, 'n_estimators': 30},
 mean: 0.90755, std: 0.06865, params: {'max_features': 1, 'n_estimators': 40},
 mean: 0.90909, std: 0.06001, params: {'max_features': 1, 'n_estimators': 50},
 mean: 0.90755, std: 0.06190, params: {'max_features': 2, 'n_estimators': 10},
 mean: 0.90293, std: 0.06630, params: {'max_features': 2, 'n_estimators': 20},
 mean: 0.90447, std: 0.06699, params: {'max_features': 2, 'n_estimators': 30},
 mean: 0.90293, std: 0.06476, params: {'max_features': 2, 'n_estimators': 40},
 mean: 0.90909, std: 0.06801, params: {'max_features': 2, 'n_estimators': 50},
 mean: 0.90601, std: 0.06493, params: {'max_features': 3, 'n_estimators': 10},
 mean: 0.90293, std: 0.07111, params: {'max_features': 3, 'n_estimators': 20},
 mean: 0.90601, std: 0.06462, params: {'max_features

In [202]:
y_pred = RandomForestClassifier(n_estimators= grid_search.best_params_['n_estimators'],
                                max_features=grid_search.best_params_['max_features'],
                                criterion='entropy', random_state=25).fit(X_train,y_train).predict(X_test)

sklearn.metrics.accuracy_score(y_test, y_pred)
sklearn.metrics.precision_score(y_test, y_pred)
sklearn.metrics.recall_score(y_test, y_pred)
sklearn.metrics.f1_score(y_test, y_pred)

0.85276073619631898

0.83116883116883122

0.85333333333333339

0.84210526315789491

In [204]:
X_test[y_test != y_pred,:], y_test[y_test != y_pred]

(array([[-0.35306336, -0.55079106,  0.72566494, -0.87471957],
        [-0.35306336, -0.55079106, -0.24056616,  0.97816467],
        [-0.35306336, -0.55079106,  0.88670345,  2.0075448 ],
        [-0.35306336, -0.55079106,  0.08151087,  0.77228864],
        [-0.35306336, -0.55079106,  1.85293455,  0.56641262],
        [-0.35306336, -0.55079106, -0.7236817 ,  0.36053659],
        [-0.35306336, -0.55079106,  0.72566494,  0.97816467],
        [-0.35306336, -0.55079106, -0.24056616, -0.25709149],
        [-0.35306336, -0.55079106,  0.72566494,  2.41929685],
        [-0.35306336, -0.55079106,  0.40358791, -0.87471957],
        [-0.35306336, -0.55079106, -0.24056616,  1.18404069],
        [-0.35306336, -0.55079106,  1.69189603,  3.44867699],
        [-0.35306336, -0.55079106,  1.04774197,  0.15466056],
        [-0.35306336, -0.55079106,  1.20878048,  0.97816467],
        [-0.35306336, -0.55079106, -0.07952764,  1.38991672],
        [-0.35306336, -0.55079106, -0.24056616, -0.66884354],
        